# Scraper de Osde ⚕

Se le pide conocer la cantidad de prestadores de la especialidad de Cirugia General para el plan 310, que hay en cada una de las siguientes provincias del Interior (tomando en cuenta todas las localidades de cada Provincia):

- Tierra del Fuego
- Entre Rios
- Cordoba
- Mendoza
- Neuquen
- Tucuman
- La Pampa

La informacion que necesita la encontrar en la siguiente pagina: [`https://www.osde.com.ar/index.html#!cartilla.html`](https://www.osde.com.ar/index.html#!cartilla.html), recuerde interactuar con la pagina para encontras las apis que necesita.

---

Usted propone la siguiente estrategia para el armado del proceso **ETL**:

### Investigacion:

- Buscar posibles APIs ocultas en la pagina que den informacion.
- Buscar si existe un parametro en la url que me permita simplemente cambiar un valor dentro de la url nada mas, y asi hacer un proceso iterativo para conseguir al informacion de todas las provincias. (Recuerda que si estas haciendo un proceso iterativo de este estilo puedes paralelizarlo).

### Extraccion:

- Llamar la API usando la libreria `requests`.
- Implementar el paralelismo usando la libreria `multiprocessing`
- Si no sabe como implementarlo siempre puede hacerlo con un for loop.

### Transformacion
- Armar un dicionario con la informacion pedida. De este estilo:
    - ```python
      resultados = {
          "provincias": ["Tierra del Fuego", "Entre Rios", ... ],
          "cantidad_de_prestadores": [23, 12, ... ],
      }
      ```
    
### Cargado (OPCIONAL):

- Armar un Dataframe con la libreria `pandas`.
- Exportar el Dataframe como un csv.

---
## NOTA IMPORTANTE SOBRE EL EJERCICIO:
El ejercicio no solo es avanzado porque te presenta la posibilidad de implementar paralelismo, tambien presenta una dificultad porque la api a usar tiene una gran cantidad de parametros, en el proceso de scrapear mediante apis ocultas tambien debemos muchas veces limpiar las urls para que solo usen la informacion minima necesaria, muchas veces hay parametros que realmente no tienen influencia sobre la respuesta final del request y deben ser eliminados de la url por motivos de limpieza en el codigo y facilidad de desarrollo. En este caso los parametros a usar son solamente:
- metodo
- rubroId
- provinciaId
- provinciaTipo
- localidadId
- planId
- especialidadId

In [ ]:
import requests
import multiprocessing as mp
import os
import pandas as pd

### Extraccion

In [ ]:
headers = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

In [ ]:
# Obtener id de los planes
url_planes = 'https://www.osde.com.ar/Cartilla/PlanRemote.ashx?metodo=ObtenerPlanesParaCartillaMedicaConNoComercial'
requests.get(url_planes, headers=headers).json()

[{'id': 21, 'nombre': '210'},
 {'id': 31, 'nombre': '310'},
 {'id': 41, 'nombre': '410'},
 {'id': 45, 'nombre': '450'},
 {'id': 51, 'nombre': '510'}]

In [ ]:
especialidad_objetivo = 411 # 'CIRUGÍA GENERAL'
plan_objetivo = 31 # '310'
provincias_objetivo = ['Tierra del Fuego', 'Entre Ríos', 'Córdoba', 'Mendoza', 'Neuquén', 'Tucumán', 'La Pampa']

url_provincias = 'https://www.osde.com.ar/Cartilla/ProvinciaRemote.ashx?metodo=ObtenerParaCartillaMedica'
response = requests.get(url_provincias, headers=headers)
prov_data = []

# Se itera por cada provincia y verifica que sea de interes
for item in response.json():
  if item['nombre'].strip() in provincias_objetivo:
    # Agregamos la informacion de interes a la lista
    prov_data.append(item['id'])
  
prov_data

[5, 7, 10, 12, 14, 22, 23]

In [ ]:
# Existe la posibilidad de ingresar los parametros GET con un diccionario
# Obtener cantidad de prestadores por 
def get_prestadores(provincia):
  short_url = 'https://www.osde.com.ar/Cartilla/consultaPorEspecialidadRemote.ashx'

  parametros = {
      'metodo': 'ObtenerParaCartillaMedica',
      'rubroId': '2', # Rubro que incluye las especialidades de cirujia
      'planId': plan_objetivo,
      'provinciaId': provincia,
      'provinciaTipo': 'INTERIOR',
      'especialidadId': especialidad_objetivo
  }

  response = requests.get(short_url, headers=headers, params=parametros)
  return response.json()['CantidadPrestadores']

#### Opcion: for loop

In [ ]:
prestadores = []
for id_prov in prov_data:
  cant_prestadores = get_prestadores(id_prov)
  prestadores.append(cant_prestadores)

prestadores

[208, 174, 39, 51, 76, 13, 92]

#### Opcion: Multiprocessing

In [ ]:
cpus = mp.cpu_count()

with mp.Pool(cpus) as pool:
  prestadores = pool.map(get_prestadores, prov_data)

Mediciones de rendimientos dependiendo la cantidad de procesos asignados, seguir en relacion a la cantidad de los cpus disponibles.

In [ ]:
%%time
with mp.Pool(cpus) as pool:
  pool.map(get_prestadores, prov_data)

CPU times: user 28.6 ms, sys: 26.9 ms, total: 55.5 ms
Wall time: 1.82 s


In [ ]:
%%time
with mp.Pool(cpus * 3) as pool:
  pool.map(get_prestadores, prov_data)

CPU times: user 30.1 ms, sys: 59.6 ms, total: 89.7 ms
Wall time: 1.14 s


In [ ]:
%%time
with mp.Pool(cpus * 4) as pool:
  pool.map(get_prestadores, prov_data)

CPU times: user 31.4 ms, sys: 72.9 ms, total: 104 ms
Wall time: 991 ms


In [ ]:
%%time
with mp.Pool(cpus * 5) as pool:
  pool.map(get_prestadores, prov_data)

CPU times: user 33.5 ms, sys: 80.3 ms, total: 114 ms
Wall time: 1.14 s


### Transformacion

In [ ]:
resultados = {
    'provincias': provincias_objetivo,
    'cantidad_de_prestadores': prestadores
}
df = pd.DataFrame.from_dict(resultados)

### Carga

In [ ]:
df.to_csv('prestadores.csv', index=False)
df

,provincias,cantidad_de_prestadores
0,Tierra del Fuego,208
1,Entre Ríos,174
2,Córdoba,39
3,Mendoza,51
4,Neuquén,76
5,Tucumán,13
6,La Pampa,92
